![Henry Logo](https://www.soyhenry.com/_next/static/media/HenryLogo.bb57fd6f.svg)

# RAG vs Agentic RAG con Batman

## Objetivo
Comparar dos arquitecturas de retrieval-augmented generation:
- **Vanilla RAG**: Retrieve -> Generate.
- **Agentic RAG**: Route -> Rewrite -> Retrieve -> Filter -> Generate -> Grounding Check.

Usamos `gpt-5-mini` para generacion y `text-embedding-3-small` para retrieval.

In [1]:
from pathlib import Path
import sys
import pandas as pd

ROOT = Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from scripts.rag_pipelines import VanillaRAG, AgenticRAG
from scripts.vector_store_lab import build_index_from_json
from scripts.evaluation import (
    build_eval_questions,
    plot_architecture_difference,
    plot_pipeline_comparison,
    run_benchmark,
)

DATA_PATH = ROOT / 'data' / 'batman_comics.json'
OUTPUTS_DIR = ROOT / 'outputs'
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

/Users/carlosdaniel/Documents/Projects/labor_projects/Henry/2026/01-introduction_ai_engineering/ai_engineering_henry/02-vector_data_bases/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db, chunks, index_stats, chunk_stats = build_index_from_json(
    json_path=DATA_PATH,
    persist_dir=OUTPUTS_DIR / 'chroma_batman_rag',
    collection_name='batman_rag_lab',
    chunk_size=800,
    chunk_overlap=120,
    embedding_model='text-embedding-3-small',
)

print(index_stats)
print(chunk_stats)

{'collection': 'batman_rag_lab', 'indexed_chunks': 38, 'embedding_provider': 'local:hash-embedding'}
{'chunk_count': 38, 'unique_sources': 12, 'avg_chars_per_chunk': 682.5, 'max_chars_per_chunk': 792, 'themes_distribution': {'origen': 3, 'villanos': 3, 'legado': 3, 'conspiracion': 3, 'derrota': 3, 'misterio': 3, 'relaciones': 3, 'equipo': 3, 'escenario': 3, 'mentoria': 4, 'recursos': 4, 'filosofia': 3}, 'hero_distribution': {'batman': 38}}


## Marco teorico: Groundedness y decisiones de parametros

### Que es groundedness (en este contexto)

La metrica de **groundedness** que usamos es un **proxy heuristico basado en token-overlap**: mide la proporcion de tokens unicos en la respuesta del LLM que tambien aparecen en los documentos de contexto recuperados.

```
groundedness = |tokens_respuesta ∩ tokens_contexto| / |tokens_respuesta|
```

**Limitaciones importantes** de este enfoque:
- **No detecta parafrasis**: si el LLM reformula una idea del contexto con sinonimos, el overlap baja aunque la respuesta sea correcta.
- **No detecta alucinaciones elaboradas**: si el LLM inventa datos usando vocabulario del contexto, el overlap sube aunque la respuesta sea incorrecta.
- **No es un LLM-judge**: metodos mas sofisticados (como NLI-based groundedness o LLM-as-judge) son mas precisos pero tambien mas costosos y lentos.

Para un laboratorio educativo, el token-overlap es un proxy util y transparente: los estudiantes pueden inspeccionar exactamente que tokens contribuyen al score, lo cual no es posible con un LLM-judge opaco.

### Por que el threshold de grounding es 0.18

En experimentos con textos narrativos en espanol (como estos comics), observamos que:
- Respuestas con **groundedness < 0.15** casi siempre contienen informacion inventada o generalizada.
- Respuestas con **groundedness > 0.25** suelen estar fuertemente ancladas al contexto.
- El rango **0.15–0.25** es una zona gris donde la calidad depende del tipo de pregunta.

El valor **0.18** es empirico, no universal. Es un punto de corte conservador que balancea:
- Evitar regeneraciones innecesarias (falsos positivos de baja calidad).
- Detectar respuestas que realmente no estan sustentadas en el contexto.

En produccion, este threshold deberia calibrarse con un dataset de evaluacion etiquetado por humanos.

### Por que Agentic RAG usa k=6 vs Vanilla RAG k=4

El pipeline Agentic RAG incluye un paso de **filtrado por relevancia** (`_filter_relevant_docs`) que elimina documentos con bajo overlap lexico con la query. Esto significa que de los `k` documentos recuperados, solo un subconjunto llega al LLM.

- **Vanilla RAG (k=4)**: todos los documentos recuperados se envian al LLM. No hay filtro, por lo que `k` debe ser conservador para no diluir el contexto con documentos irrelevantes.
- **Agentic RAG (k=6)**: el pool inicial es mas grande porque el filtrado posterior reduce la cantidad. Si el filtro elimina 2-3 documentos, aun quedan 3-4 relevantes para la generacion.

Este es un patron comun en pipelines RAG de produccion: **over-retrieve + filter > retrieve exacto**.

In [3]:
vanilla = VanillaRAG(
    vector_db=db,
    model='gpt-5-mini',
    embedding_model='text-embedding-3-small',
    k=4,
)

agentic = AgenticRAG(
    vector_db=db,
    model='gpt-5-mini',
    embedding_model='text-embedding-3-small',
    k=6,
    min_docs_after_filter=3,
)

print('Pipelines initialized.')

Pipelines initialized.


In [4]:
query = 'Compara como Batman enfrenta a Bane en Knightfall versus su enfoque contra el Joker en The Killing Joke.'

vanilla_result = vanilla.run(query)
agentic_result = agentic.run(query)

comparison_preview = pd.DataFrame([
    {
        'pipeline': vanilla_result.pipeline,
        'latency_seconds': vanilla_result.latency_seconds,
        'groundedness': vanilla_result.groundedness,
        'retrieved_docs': len(vanilla_result.docs),
        'route': vanilla_result.route,
        'llm_provider': vanilla_result.llm_provider,
    },
    {
        'pipeline': agentic_result.pipeline,
        'latency_seconds': agentic_result.latency_seconds,
        'groundedness': agentic_result.groundedness,
        'retrieved_docs': len(agentic_result.docs),
        'route': agentic_result.route,
        'llm_provider': agentic_result.llm_provider,
    },
])
comparison_preview

,pipeline,latency_seconds,groundedness,retrieved_docs,route,llm_provider
0,vanilla_rag,0.0021,0.9041,4,n/a,local:fallback-summary
1,agentic_rag,0.0008,0.9351,6,villains,local:fallback-summary


## Diferencia estructural de pipelines

```mermaid
flowchart TD
  A["User Query"] --> B["Retrieve"] --> C["Generate"]

  D["User Query"] --> E["Route"] --> F["Rewrite"] --> G["Retrieve"] --> H["Filter"] --> I["Generate"] --> J["Grounding Check"]
```

In [5]:
questions = build_eval_questions()
benchmark_df = run_benchmark(vanilla=vanilla, agentic=agentic, queries=questions)
benchmark_df.head()

,pipeline,query,latency_seconds,retrieved_docs,route,groundedness_proxy,groundedness_internal,llm_provider,retrieval_provider,steps
0,vanilla_rag,Explica como evoluciona Batman desde Year One ...,0.0006,4,n/a,0.8667,0.8667,local:fallback-summary,local:hash-embedding,retrieve -> generate
1,agentic_rag,Explica como evoluciona Batman desde Year One ...,0.0007,6,chronology,0.8519,0.8519,local:fallback-summary,local:hash-embedding,route -> rewrite_query -> retrieve -> generate
2,vanilla_rag,Cual es la tesis filosofica central de The Kil...,0.0005,4,n/a,0.9194,0.9194,local:fallback-summary,local:hash-embedding,retrieve -> generate
3,agentic_rag,Cual es la tesis filosofica central de The Kil...,0.0007,6,lore,0.9194,0.9194,local:fallback-summary,local:hash-embedding,route -> rewrite_query -> retrieve -> generate
4,vanilla_rag,Compara la estrategia de Bane en Knightfall co...,0.0005,4,n/a,0.9079,0.9079,local:fallback-summary,local:hash-embedding,retrieve -> generate


In [6]:
summary_df = plot_pipeline_comparison(
    benchmark_df,
    output_path=OUTPUTS_DIR / 'rag_vs_agentic_rag_metrics.png',
)
plot_architecture_difference(OUTPUTS_DIR / 'rag_vs_agentic_architecture.png')
summary_df

,pipeline,latency_seconds,groundedness_proxy,retrieved_docs
0,agentic_rag,0.0006,0.8773,6.0
1,vanilla_rag,0.0005,0.8751,4.0


In [7]:
csv_path = OUTPUTS_DIR / 'rag_vs_agentic_benchmark.csv'
benchmark_df.to_csv(csv_path, index=False)
print(f'Saved benchmark rows: {len(benchmark_df)}')
print(f'CSV: {csv_path}')
print(f'Metrics plot: {OUTPUTS_DIR / "rag_vs_agentic_rag_metrics.png"}')
print(f'Architecture plot: {OUTPUTS_DIR / "rag_vs_agentic_architecture.png"}')

Saved benchmark rows: 16
CSV: /Users/carlosdaniel/Documents/Projects/labor_projects/Henry/2026/01-introduction_ai_engineering/ai_engineering_henry/02-vector_data_bases/batman_vector_db_orchestration/outputs/rag_vs_agentic_benchmark.csv
Metrics plot: /Users/carlosdaniel/Documents/Projects/labor_projects/Henry/2026/01-introduction_ai_engineering/ai_engineering_henry/02-vector_data_bases/batman_vector_db_orchestration/outputs/rag_vs_agentic_rag_metrics.png
Architecture plot: /Users/carlosdaniel/Documents/Projects/labor_projects/Henry/2026/01-introduction_ai_engineering/ai_engineering_henry/02-vector_data_bases/batman_vector_db_orchestration/outputs/rag_vs_agentic_architecture.png


## Interpretacion orientada a ingenieria

- Si Agentic RAG mejora groundedness sin deteriorar demasiado latencia, suele ser la opcion de produccion para consultas complejas.
- Si el dominio es muy cerrado y estable, Vanilla RAG puede ser suficiente y mas barato.
- El punto clave no es "agentes por moda", sino el costo-beneficio medible por caso de uso.

### Que mirar en los resultados del benchmark

- **Groundedness proxy**: compara las medias entre pipelines. Si Agentic RAG no supera a Vanilla, el overhead de routing/filtering no se justifica.
- **Latencia**: Agentic RAG sera siempre mas lento (mas pasos). La pregunta es si el delta de latencia es aceptable para la ganancia en calidad.
- **Steps ejecutados**: revisa si `regenerate_if_low_grounding` se activa frecuentemente. Si lo hace, indica que el retrieval inicial necesita mejora (mejor chunking, mejor query rewriting, o mas documentos).

## Cierre didactico

### Conceptos clave de esta notebook

- **Vanilla RAG es un baseline, no un strawman**: su simplicidad es una ventaja real en produccion (menos puntos de falla, menor latencia, mas facil de debuggear).
- **Agentic RAG agrega valor solo cuando sus pasos adicionales producen mejoras medibles**: routing sin beneficio, filtering que no filtra, o grounding checks con thresholds mal calibrados son overhead puro.
- **Groundedness basado en token-overlap es un proxy pragmatico**, no una metrica de evaluacion definitiva. En produccion, se complementa con LLM-judges, evaluacion humana, o metricas de NLI (Natural Language Inference).
- **El parametro `k` no es un numero magico**: depende de la estrategia del pipeline. Over-retrieve + filter es un patron valido cuando el filtrado agrega valor.

### Conexion con la siguiente notebook

En la siguiente notebook aplicamos **routing entre dominios** (Batman vs Spider-Man) para ver como un orquestador simple puede dirigir consultas al pipeline correcto. La pregunta central: *cuando routing heuristico es suficiente y cuando necesitas routing semantico?*